In [1]:
import numpy as np
import nltk
import math
from sklearn.preprocessing import normalize

In [2]:
def open_file():
    query = {}
    doc = {}
# ntust-ir-2020
    with open('./query_list.txt', 'r') as file:
        query_list = file.read()
        for file_name in query_list.split('\n'):
            try:
                file_path = f'./queries/{file_name}.txt'
                with open(file_path, 'r') as f:
                    query[file_name] = f.read().lower()
            except Exception as e:
                print(e)

    with open('./doc_list.txt', 'r') as file:
        doc_list = file.read()
        for file_name in doc_list.split('\n'):
            try:
                file_path = f'./docs/{file_name}.txt'
                with open(file_path, 'r') as f:
                    doc[file_name] = f.read().lower()
            except Exception as e:
                print(e)

    print(len(query))
    print(len(doc))
    
    return query, doc

In [3]:
def cal_df(doc_dict):
    word_list = {}
    all_word = []
    
    for doc_name, doc in doc_dict.items():
        word_list[doc_name] = {}
        for word in set(doc.split(' ')):
            word_list[doc_name][word] = ''
            all_word.append(word)
        all_word.extend(list(set(word_list)))

    df_dict = {}
    for word in list(set(all_word)):      # 計算 df
        freq = 0
        for doc_name in word_list.keys():
            if word_list[doc_name].get(word, ''):
                freq = freq + 1

        df_dict[word] = freq
        
    return df_dict

In [6]:
def cal(df_dict, doc_dict, N):
    tf_dict = {}   # tf[doc][word]
    idf_dict = {}  # idf[word]
    
    for doc_name, doc in doc_dict.items():
        tf_dict[doc_name] = {}
        for word in doc.split(' '):
            if tf_dict[doc_name].get(word, 0):  # 計算 tf
                tf_dict[doc_name][word] += 1
            else:
                tf_dict[doc_name][word] = 1
    
    for word, df_freq in df_dict.items():  # 計算 idf
        idf_dict[word] = math.log(((N+1) / (df_freq+1)) + 1)  # smooth 公式
        
    return tf_dict, idf_dict

In [7]:
def cal_tf_idf(tf, idf, doc_dict):
    tf_idf_dict = {}  # tf_idf[doc][word]
    
    for doc_name, doc in doc_dict.items():
        tf_idf_dict[doc_name] = {}
        for word in doc.split(' '):
            tf_idf_dict[doc_name][word] = (1 + math.log(tf[doc_name][word])) * idf[word]
    # L2 normalize
#     tf_idf_dict = normalize( )
    return tf_idf_dict 

In [8]:
# from sklearn.metrics.pairwise import cosine_similarity

def vectorSpaceModel(query, doc_dict, tf_idf, idf):
    query_vec = [idf[word] if idf.get(word, 0) else 0 for word in query.split(' ')] # 將 word 轉成 score
    score_dict = {}
#     print(query_vec)
    
    for doc_name, doc in doc_dict.items():
        doc_vec = [tf_idf[doc_name][word] if tf_idf[doc_name].get(word, 0) else 0 for word in query.split(' ')] # 將 word 轉成 score
        score_dict[doc_name] = np.dot(query_vec, doc_vec) / len(query_vec)
#         print(doc_vec)
#         score_dict[doc_name] = cosine_similarity([query_vec], [doc_vec])
#         print(score_dict[doc_name])
    
    rank = sorted(score_dict.items(), key=lambda x: x[1], reverse = True)
    return rank

In [14]:
query_dict, doc_dict = open_file()
print(query_dict['301'].split())

[Errno 2] No such file or directory: './queries/.txt'
[Errno 2] No such file or directory: './docs/.txt'
50
4191
['intern', 'organ', 'crime']


In [21]:
all_voc = {} ## 字典
all_split_list = [] ##所有斷詞後的結果
for i in (doc_dict.values()):
    all_split_list.append(i.split())
    
## 初始化字典
for i in range(len(all_split_list)):
    for j in range(len(all_split_list[i])):
        all_voc[all_split_list[i][j]] = 0.0
print(all_voc['crime'])

0.0


In [10]:
word_list = {}
all_word = []
all_voc = {}
for doc_name, doc in doc_dict.items():
    word_list[doc_name] = {}
    for word in set(doc.split(' ')):
        word_list[doc_name][word] = ''
        all_word.append(word)
    all_word.extend(list(set(word_list)))
print(all_word)
df_dict = {}
for word in list(set(all_word)):      # 計算 df
    freq = 0
    for doc_name in word_list.keys():
        if word_list[doc_name].get(word, ''):
            freq = freq + 1

    df_dict[word] = freq
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
df = cal_df(doc_dict)
print(df['crime'])
tf, idf = cal(df, doc_dict, len(doc_dict))

0


In [72]:
tf_idf = cal_tf_idf(tf, idf, doc_dict)

In [73]:
f = open('ans.txt', 'w')
string = 'Query,RetrievedDocuments\n'

for _id, _query in query_dict.items():
    rank = vectorSpaceModel(_query, doc_dict, tf_idf, idf)

    string += _id + ','
    for doc in rank:
        string += doc[0] + ' '
    string += '\n'
    
f.write(string)
f.close()
print('done')

done
